In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snsdata_raw = pd.read_csv(‘https://github.com/Thomas-Richardson/Blog_post_data/blob/main/UFC_data_raw_2021-03-11.csv?raw=true')data_clean = data_raw

SyntaxError: invalid character '‘' (U+2018) (4023270967.py, line 4)

In [ ]:
data_clean.head(20)

In [ ]:
data_clean.info()

In [ ]:
data_clean = data_clean.drop(['Unnamed: 0', 'Belt'], axis = 1)

In [ ]:
data_clean.columns = [‘first_name’, ‘last_name’, ‘nickname’, ‘height’, ‘weight_lbs’, ‘armspan_inches’, ‘stance’, ‘wins’, ‘losses’, ‘draws’]

In [ ]:
data_clean.loc[data_clean.armspan_inches =='--','armspan_inches'] = np.nandata_clean.armspan_inches = data_clean.armspan_inches \
.str.replace('\"','') \
.astype('float')sns.histplot(data = data_clean,
x = 'armspan_inches',
color = 'navy')plt.xlabel('fighter armpan in inches')
plt.show()

In [ ]:
data_clean.height = data_clean.height.str.replace('\"','')data_clean.loc[data_clean.height == '--','height'] = np.nandata_clean.loc[:,'height_split'] = data_clean.height.str.split('\' ')data_clean.loc[:,'height_split']

In [ ]:
data_clean.loc[:,'height_ft'] = [row[0] if isinstance(row,list) else np.nan for row in data_clean.height_split]

data_clean.loc[:,'height_inches'] = [row[1] if isinstance(row,list) else np.nan for row in data_clean.height_split]

In [ ]:
data_clean.loc[:,'height'] = data_clean.loc[:,'height_inches'].astype('float')+ (data_clean.loc[:,'height_ft'].astype('float')*12)data_clean = data_clean.drop(columns = ['height_inches','height_ft','height_split']).rename(columns = {'height':'height_inches'})sns.histplot(data = data_clean, x = 'height_inches', color = 'navy',bins = 15)plt.xlabel('fighter height in inches')
plt.show()

In [ ]:
data_clean.loc[data_clean.weight_lbs =='--','weight_lbs'] = np.nan
data_clean.weight_lbs= data_clean.weight_lbs.str.replace(' lbs','').astype('float')

sns.histplot(data = data_clean, x = 'weight_lbs', color = 'navy', bins = 30)
plt.xlabel('fighter weight in lbs')
plt.show()

In [ ]:
data_clean.weight_lbs.sort_values(ascending = False)

In [ ]:
data_clean.loc[data_clean.weight_lbs==770,:]

In [ ]:
without_emmanuel = data_clean.loc[data_clean.weight_lbs <770,:]

sns.histplot(data = without_emmanuel, x = 'weight_lbs', color = 'navy', binwidth=12)
plt.xlabel('fighter weight in lbs')
plt.show()

In [ ]:
data_clean = data_clean.fillna('Missing')
data_clean.value_counts('stance')

In [ ]:
sns.countplot(y = 'stance', data = data_clean, order = ['Orthodox', 'Southpaw','Switch','Open Stance','Sideways','Missing'])
plt.show()

In [ ]:
data_clean.loc[:,'total_fights'] = data_clean.loc[:,['wins','draws','losses']].sum(axis = 1)

In [ ]:
mapping = {'Open Stance':'Mixed','Sideways':'Mixed','Switch':'Mixed'}
data_clean.loc[:,'stance2'] = data_clean.loc[:,'stance'].replace(mapping)

sns.countplot(y = 'stance2', data=data_clean, order = ['Orthodox','Southpaw','Mixed','Missing'])

plt.show()

In [ ]:
#write.csv(data_clean, ’UFC_data_cleaned.csv’)

In [ ]:
import statsmodels.api as sm
data = data_clean
data = data.drop(axis = 1, columns = ['Unnamed: 0'])

In [ ]:
data.hist(grid = False, bins = 15)
plt.show()

In [ ]:
data.loc[data.total_fights > 100,:]

In [ ]:
correlation_matrix = data.loc[:,['height_inches', 'weight_lb', 'armspan_inches', 'losses', 'wins','total_fights']].corr(method = 'spearman').round(2)

mask = np.zeros_like(correlation_matrix)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(correlation_matrix, annot=True,vmin = -1, vmax=1,mask=mask, square=True, cmap="BuPu")
plt.xticks(rotation = 60)
plt.yticks(rotation = 0)
plt.show()

In [ ]:
data.loc[:,'win_percentage'] = (data.loc[:,'wins']+ (0.5*data.loc[:,'draws']) )/data.loc[:,'total_fights']

sns.histplot(data = data, x = 'win_percentage', color = 'navy', bins = 20)
plt.show()

In [ ]:
data_model = data.loc[:, [‘armspan_inches’, ’height_inches’, ’weight_lb’]].dropna()

In [ ]:
X = data_model.loc[:,['height_inches','weight_lb']]
X = sm.add_constant(X)
y = data_model.loc[:,'armspan_inches']

sm.OLS(y, X).fit().summary()

In [ ]:
data_model2 = data.loc[:,['armspan_inches','height_inches','weight_lb', 'win_percentage']].dropna()

X = data_model2.loc[:,['height_inches','weight_lb','armspan_inches']]
X = sm.add_constant(X)
y = data_model2.loc[:,'win_percentage']
sm.OLS(y,X).fit().summary()

In [ ]:
data.loc[data.weight_lb == 155,'armspan_inches'].agg(['min','max'])